# Step 0.1. Import necessary libraries 

In [ ]:
# Standard python libraries
import logging
import os
import time

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, median_absolute_error
from sklearn.model_selection import train_test_split
import torch
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

from tqdm.notebook import tqdm

from lightautoml.tasks import Task
from lightautoml.addons.autots.base import AutoTS
from lightautoml.dataset.roles import DatetimeRole

In [ ]:
X_train = pd.read_csv('ts_dataset_1id.csv')
X_train['date'] = pd.to_datetime(X_train.date)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot('date', 'stage_max', data=X_train)
fmt_half_year = mdates.MonthLocator(interval=12)
ax.xaxis.set_major_locator(fmt_half_year)
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.format_xdata = mdates.DateFormatter('%Y-%m')

fig.autofmt_xdate()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot('date', 'stage_max', data=X_train[X_train.date >= '2018-01-01'], linestyle='none', marker='o')
fmt_half_year = mdates.MonthLocator(interval=1)
ax.xaxis.set_major_locator(fmt_half_year)
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.format_xdata = mdates.DateFormatter('%Y-%m')

fig.autofmt_xdate()
plt.show()

In [ ]:
col = X_train[X_train.date >= '2018-01-01'].date

In [ ]:
seq_params = {'seq0': {'case': 'next_values', # тип задачи, которую решаем - предсказание следующих значений ряда
                           'params': {'n_target': 7, # число точек, на сколько вперед предсказываем
                                      'history': 7, # размер истории для подсчета признаков
                                      'step': 1, # шаг, через сколько точек взять следующие наблюдение
                                      'from_last': True, # датасет заканчивается на последнем доступном наблюдении 
                                      'test_last': False # предсказывать только последний участок из теста
                                     }},}
trend_params = {'trend': False}

In [ ]:
%%time

# трейн датасет
train = X_train.iloc[:-len(col)].copy().fillna(0)
# тест датасет
# +7 точек пересечения с трейном для подсчета признаков на первую дату теста
test = X_train.iloc[-(len(col)+7):].copy().fillna(0)

roles = {'target': 'stage_max',
         DatetimeRole(seasonality=('d', 'm', 'doy')): 'date'}
task = Task('multi:reg', greater_is_better=False, metric='mae', loss='mae')
model = AutoTS(task, seq_params=seq_params, trend_params=trend_params)

oof_pred_seq, median = model.fit_predict(train, roles)
pred, trend = model.predict(test)

In [ ]:
# структура предсказаний
# pred:
# pred[0] - предсказание на первых history теста,
# в нашем случае - с признаков последней недели трейна на первую неделю теста
# pred[-1] - предсказание последние history теста,
# в нашем случае это признаки последних 7 дней тестовых данных на неизветное для нас будущее.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(col, test.stage_max.values[7:], c='b', label='Ground Truth')
ax.plot(col, pred[:-1, 0], c='r', label='1st day prediction')
ax.plot(col, [np.NaN] * 1 + list(pred[:-2, 1]), c='k', label='2st day prediction')
ax.plot(col, [np.NaN] * 6 + list(pred[:-7, 6]), c='g', label='7st day prediction')
plt.legend()
plt.show()

In [ ]:
# на текущий момент не хотим считать метрику по 7 наблюдениям перед обрывом в наблюденяих,
# так как там неправильно собирается целевая переменная и предикт (на данных с июня предсказываем ноябрь)
COND = ((col <= '2018-06-23') | (col > '2018-06-30')).values

In [ ]:
for i in range(7):
    print("Day {}, MAE: {}".format(i,
                                   np.round(median_absolute_error(test[roles['target']].values[7:][COND][i:],
                                                                  pred[:-1][COND][:-i, i] if i!= 0 else pred[:-1][COND][:, i]), 4)))

## Сравнение с бейзлайнами:

### Медиана по дню недели.

In [ ]:
train_c = train[['date', 'stage_max']].copy()
test_c = test[['date', 'stage_max']].iloc[7:].copy()
for df in [train_c, test_c]:
    df['dayofyear'] = pd.to_datetime(df.date).dt.dayofyear
    
statistic = train_c.groupby('dayofyear')['stage_max'].apply(np.median)
pred_naive = test_c.dayofyear.map(statistic)
print("Naive, MAE: {}".format(np.round(median_absolute_error(test_c[roles['target']].values[COND], pred_naive[COND]), 4)))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(col, test.stage_max.values[7:], c='b', label='Ground Truth')
ax.plot(col, pred_naive.values, c='r', label='prediction')

plt.legend()
plt.show()

### Значение за предыдущий год.

In [ ]:
train_c = train[['date', 'stage_max']].copy()
train_c = train_c[train_c.date >= '2017-01-01']
test_c = test[['date', 'stage_max']].iloc[7:].copy()
for df in [train_c, test_c]:
    df['dayofyear'] = pd.to_datetime(df.date).dt.dayofyear
    
statistic = train_c.groupby('dayofyear')['stage_max'].apply(np.median)
pred_naive = test_c.dayofyear.map(statistic)
print("Naive, MAE: {}".format(np.round(median_absolute_error(test_c[roles['target']].values[COND], pred_naive[COND]), 4)))

### Эвристика с ТОП2 private lb хакатона.

In [ ]:
train_c = train[['date', 'stage_max']].copy()
test_c = test[['date', 'stage_max']].iloc[7:].copy()
for df in [train_c, test_c]:
    df['year'] = pd.to_datetime(df.date).dt.year
    df['dayofyear'] = pd.to_datetime(df.date).dt.dayofyear


df_avg = []
for pair, group in train_c.groupby(['year']):
    avg_at_year = pd.concat([group['stage_max'].shift(i) for i in range(-10, 10)], axis=1).mean(axis=1)
    avg_at_year = pd.DataFrame(avg_at_year, columns=['stage_max_avg'])
    avg_at_year['year'] = pair
    avg_at_year['date'] = group['date']
    df_avg.append(avg_at_year)
df_avg = pd.concat(df_avg)
df_avg['dayofyear'] =  pd.to_datetime(df_avg['date']).dt.dayofyear
statistic2 = df_avg.groupby(['dayofyear'])['stage_max_avg'].median()

pred_naive2 = test_c.dayofyear.map(statistic2)
print("Naive 2, MAE: {}".format(np.round(median_absolute_error(test_c[roles['target']].values[COND], pred_naive2[COND]), 4)))


In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(col, test.stage_max.values[7:], c='b', label='Ground Truth')
ax.plot(col, pred_naive2.values, c='r', label='prediction')

plt.legend()
plt.show()

### Медиана в скользящем окне размера 7.

In [ ]:
train_c = train[['date', 'stage_max']].copy()
test_c = test[['date', 'stage_max']].copy()

pred_rolling = test_c.rolling(7).median().dropna().values.flatten()[:-1]    
for i in range(7):
    print("Day {}, MAE: {}".format(i,
                                   np.round(median_absolute_error(test[roles['target']].values[7:][COND][i:],
                                                                  pred_rolling[COND][:-i] if i!=0 else pred_rolling[COND]), 4)))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(col, test.stage_max.values[7:], c='b', label='Ground Truth')
ax.plot(col, pred_rolling, c='r', label='1st day prediction')
ax.plot(col, [np.NaN] * 6 + list(pred_rolling[:-6]), c='g', label='7st day prediction')
plt.legend()
plt.show()

### Предсказание последним известным значением.

In [ ]:
def get_last(x):
    return list(x)[-1]

train_c = train[['date', 'stage_max']].copy()
test_c = test[['date', 'stage_max']].copy()

pred_rolling_last = test_c.rolling(7).apply(get_last).dropna().values.flatten()[:-1]
for i in range(7):
    print("Day {}, MAE: {}".format(i,
                                   np.round(median_absolute_error(test[roles['target']].values[7:][COND][i:],
                                                                  pred_rolling_last[COND][:-i] if i!=0 else pred_rolling_last[COND]), 4)))

#### Еще раз метрики модели

In [ ]:

for i in range(7):
    print("Day {}, MAE: {}".format(i,
                                   np.round(median_absolute_error(test[roles['target']].values[7:][COND][i:],
                                                                  pred[:-1][COND][:-i, i] if i!= 0 else pred[:-1][COND][:, i]), 4)))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(col, test.stage_max.values[7:], c='b', label='Ground Truth')
ax.plot(col, pred_rolling_last, c='r', label='1st day prediction')
ax.plot(col, [np.NaN] * 6 + list(pred_rolling_last[:-6]), c='g', label='7st day prediction')
plt.legend()
plt.show()